In [ ]:
!pip install pytesseract 
!apt-get install poppler-utils 
!pip install pytesseract
!pip install opencv-python
!pip install pandas

In [ ]:
!pip install pdf2image --user
!pip install pytesseract --user

In [12]:
import pdf2image
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
from pytesseract import image_to_string
from IPython.display import Image as pim, display
import csv
import os
import pandas as pd
from multiprocessing import Pool

In [13]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [14]:
CORES = 30
DIRECTORY_PATH = "../Delhi/"
DOWNLOAD_LOCATION = "../ParsedData/Delhi/"
LOG_DIR = "../ParsedData/Delhi/logs/"
TEMP_DIR = "../ParsedData/Delhi/temp/"

In [15]:
def convert_to_csv(filename, data, header):
    df = pd.DataFrame(data, columns = header)
    filename = filename.replace(".pdf","")
    path = DOWNLOAD_LOCATION+filename+"_parsed.csv"
    with open(LOG_DIR+'log_ac_left.txt', 'a') as log_file:
        log_file.write(f"PID-{os.getpid()} at {path} !!\n")
    df.to_csv(path)

def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path(pdf_file)

def ocr_core(file):
    text = pytesseract.image_to_string(file)
    return text

def get_name(line_str, rem_str):
    line_str = line_str.replace(rem_str, "")
    line_str = line_str.strip()
    # Assuming first letter of a name to be alphabet
    i = 0
    for i, char in enumerate(line_str):
      if char.isalpha():
        break
    line_str = line_str[i:]
    return line_str

def parse_info(elem, data):
    husband_name = ""
    father_name = ""
    mother_name = ""
    house_number = ""
    name = ""
    age = 0
    gender = ""
    for line in elem.split('\n'):
      # print(line)
      if "Husband's Name" in line:
        husband_name = get_name(line, "Husband's Name")
      elif "Father's Name" in line:
        father_name = get_name(line, "Father's Name")
      elif "Mother's Name" in line:
        mother_name = get_name(line, "Mother's Name")
      elif "House Number" in line:
        house_number = get_name(line, "House Number")
      elif "Name" in line:
        name = get_name(line, "Name")
      elif "Age" in line:
        try:
          age = int(line[5:7])
        except:
          pass
        # Considering only 2 genders [for now]
        if "FEMALE" in line:
            gender = "FEMALE"
        elif "MALE" in line:
            gender = "MALE"
      elif "FEMALE" in line:
        gender = "FEMALE"
      elif "MALE" in line:
        gender = "MALE"
    if name == "":
      return data
    arr = [name, father_name, husband_name, mother_name, house_number, gender, age]
#     print(husband_name,":",father_name,":",mother_name,":",house_number,":", name,":",str(age),":",gender)
    data.append(arr)
    return data

def extract_to_csv(img, data):
    pid = str(os.getpid())   
#     with open(LOG_DIR+'log_ac1_5.txt', 'a') as log_file:
#         log_file.write(f"{os.getpid()}:")
    image_name = pid+"sample_delhi.jpeg"  
    img.save(TEMP_DIR+image_name, "JPEG")
    # Extract data from page and delete
    # Highlight the region that we want to extract
    upper = 140
    lower = 333
    d1 = lower-upper
    for i in range(1,11):
      right = 565
      left = 50
      d = right - left
      for j in range(1,4):
          im = Image.open(TEMP_DIR+image_name)
          im_crop = im.crop((left, upper, right, lower))
          im_crop.save(TEMP_DIR+pid+'test_delhi.png', quality=95)
          # display(pim('test.png'))
          text = ocr_core(Image.open(TEMP_DIR+pid+'test_delhi.png'))
          data = parse_info(text, data)
          # print(text)          
          left = left+d
          right = right+d
      upper = upper + d1
      lower = lower + d1
    return data

def convert_pdf_to_csv(pdf_file):
    header = ['name', 'father_name', 'husband_name', 'mother_name', 'house_number', 'gender', 'age']
    data = []
    loc = DIRECTORY_PATH
    try:
        images = pdf_to_img(loc+pdf_file)

        with open(LOG_DIR+'log_ac_left.txt', 'a') as log:
            log.write(f"PID- {os.getpid()} : is converting {pdf_file} ==== \n")
        for img in images[2:-1]:
            data = extract_to_csv(img, data)
        convert_to_csv(pdf_file, data, header)
    except:
        with open(LOG_DIR+'log_wrong.txt', 'a') as log_file:
            log_file.write(f"Unable to parse {pdf_file} by {os.getpid()}\n")

# Convert all the pdfs present in a folder to a csv
def extract_data(files):
    for file in files:
        convert_pdf_to_csv(file)

In [16]:
# Get all files in a directory
def get_files(location):
    direc = os.listdir(location)
    arr = []

    for elem in direc:
        if ".pdf" in elem:
#             if "AC62_" in elem or "AC63_" in elem or "AC64_" in elem or "AC65_" in elem or "AC66_" in elem or "AC67_" in elem or "AC68_" in elem or "AC69_" in elem or "AC70_" in elem:
                arr.append(elem)
        
    return arr

In [17]:
def del_downloaded_files(location, files):
    downloaded_files = os.listdir(location)
    rest = []
    for elem in files:
        temp = elem
        elem = elem.replace(".pdf","")
        elem = elem + "_parsed.csv"
        if elem not in downloaded_files:
            rest.append(temp)
    return rest

In [18]:
len(get_files(DIRECTORY_PATH))

14462

In [19]:
files = del_downloaded_files(DOWNLOAD_LOCATION,get_files(DIRECTORY_PATH))

In [20]:
print(len(files))

756


In [21]:
# print(files)

In [ ]:
with Pool(CORES) as p:
    p.map(convert_pdf_to_csv, files)